## Generate Stateset for the day

In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
from custom_pipeline import get_concs, get_stateset, scale_stateset
from pathlib import Path
import sys
import pickle
package_path = Path.cwd().parent
if package_path not in sys.path:
    sys.path.append(str(package_path))
from utils.dataset_class import Phase3DataSet, Setting

In [22]:
stateset_raw = pd.read_csv('../phase3/final_data.csv')
chemical_amounts = pd.read_csv('./extra_data/al_ingredients.csv')

In [23]:
actual_amounts = {
    '_raw_reagent_0_chemicals_0_actual_amount' : 3.78, #ml    - DMF     - Reagent 1
    
    '_raw_reagent_1_chemicals_0_actual_amount' : 3.54, #grams - PbI2    - Reagent 2
    '_raw_reagent_1_chemicals_1_actual_amount' : 1.33, #grams - Me2NH2I - Reagent 2
    '_raw_reagent_1_chemicals_2_actual_amount' : 2.90, #ml    - DMF     - Reagent 2
    
    '_raw_reagent_2_chemicals_0_actual_amount' : 1.35, #grams - Me2NH2I - Reagent 3
    '_raw_reagent_2_chemicals_1_actual_amount' : 2.72, #ml    - DMF     - Reagent 3
    
    '_raw_reagent_6_chemicals_0_actual_amount' : 2.72, #ml    - FAH     - Reagent 5
    
}
for chemical in actual_amounts:
    chemical_amounts.loc[0, chemical] = actual_amounts[chemical]

In [24]:
stateset_template = pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')
ss_reagent_vols = stateset_template[[col for col in stateset_template.columns if 'Reagent' in col]]
chemdf_dict = {'HC': pd.read_csv('./extra_data/chemical_inventory.csv', index_col='InChI Key (ID)')}
phase3_training = pickle.load(open('phase3_dataset.pkl', 'rb'))

/Users/vshekar/anaconda3/envs/escalate/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [25]:
summed_mmol, summed_molarity = get_concs(chemical_amounts, chemdf_dict, ss_reagent_vols)
summed_mmol.index.rename('name')
summed_molarity.index.rename('name')
stateset = get_stateset(summed_mmol, summed_molarity)
x_stateset = scale_stateset(stateset, phase3_training.scaler)

## Write today's stateset to disk

In [30]:
stateset.to_csv(f'./statesets/stateset_{datetime.today().date()}.csv')
np.savetxt(f'./statesets/scaled_stateset_{datetime.today().date()}.csv', x_stateset, delimiter=",")